In [2]:
wd <- dirname(getwd())
source(paste0(wd,"/mission_control/treasure_map.R"))
#source(paste0(wd,"/mission_control/isofox_help.R"))

library(dplyr)
library(tidyverse)
library(data.table)

#args <- commandArgs(trailing = TRUE) ## "RawTPM" or "AdjTPM" #
args <- list("AdjTPM")

### Read in Hartwig Genes

In [6]:
isofox <- fread(paste0( TMP_DIR,"isofox_",args[1],"_features.csv"))
isofox_genes <- unlist(lapply(colnames(isofox)[-1], function(i) strsplit(i, "_")[[1]][2]))

### Read in TGFB sets and explore

In [7]:
tgfb_sets <- read.csv(paste0(REF_DIR, "battle_tgfb_sets.csv"), sep = ";", stringsAsFactors = FALSE)
tgfb_sets <- lapply(tgfb_sets, toupper)

In [8]:
battle_all <- unique(unlist(tgfb_sets))

In [9]:
missing <- battle_all[-which(battle_all %in% isofox_genes)]

In [12]:
missing_map <- list(
    'ANK' = 'ANK1',
    'SAA3' = NULL,
    'C1S1' = 'C1S',
    'LY6A' = NULL,
    'AES' = 'TLE5',
    'IFI27L2A' = 'IFI27L2',
    'AY036118' = NULL, 
    'GM156' = NULL, 
    'GM30054' = NULL,
    'GM42418' = NULL, 
    'GM47015' = NULL, 
    'LY6C2' = NULL, 
    'MS4A4B' = NULL,
    'MS4A6B' = NULL
)

In [13]:
clean_list <- function( ll ){
    clean_list <- c()
    for( i in ll ){
        if(i != ""){
            if( i %in% names(missing_map) ){ 
                i <- missing_map[[i]] 
            }
            clean_list <- c(clean_list,i)
        }
    }
    clean_list
}
tidy_battle <- list()
for( i in names(tgfb_sets)){
    tidy_battle[[i]] <- clean_list(tgfb_sets[[i]])
}

In [15]:
saveRDS(tidy_battle, paste0( REF_DIR,"battle_gene_set.Rds"))

##### Create gene set ref

In [31]:
gene_map <- list()
for( i in names(gene_sets)){
    gene_map[[i]] <- surgeon(i)
    for (j in gene_sets[[i]]){
        nice_name <- surgeon(j)
        if( length(gene_map[[j]]) == 0){
            gene_map[[j]] <- surgeon(i)
        }
    }
}
gene_map <- data.frame( gene_set_typ = t(data.frame(gene_map))); 
gene_map$feature <- rownames(gene_map); 
gene_map <- gene_map %>% relocate(feature)
rownames(gene_map) <- NULL

In [32]:
saveRDS(gene_map, paste0(REF_DIR,"gene_set_map.Rds"))

#### 3 - Format for Univariate Scan

In [33]:
happy_isofox <- isofox2 %>% left_join(gene_sets_final, by = "sampleId")
rownames(happy_isofox) <- NULL

#### 4 - Save for scan analyses

In [33]:
fwrite( happy_isofox, file= paste0( TMP_DIR, "isofox_", args[1], "_ready.csv") )